In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Search

> Functionalities to search, and retrieve data from pubmed

In [ ]:
#| default_exp search

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from Bio import Entrez
import os
from datetime import datetime, timedelta, date
from fastcore.all import *
from typing import Union, Optional
from pydantic import BaseModel, ValidationError, validator

In [ ]:
#| export
from pubmed_lib.data import *
from pubmed_lib.result import *
from pubmed_lib.parser import *

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
load_dotenv('pass.env')
email = os.environ.get('EMAIL')
api_key = os.environ.get('API_KEY')

In [ ]:
#| exports

class Search(BaseModel):
    """
    Search class to warp the search and results
    """
    search_tag:str = None #Tag to specifiy the search, can be any from pubmed, Defaul: Title/Abstract
    retmax:int = 200 #Maximum number of results to be retrieved
    retmode:str ='xml' #Format of the returned data, options are xml, 
    sort:str='relevance' #Way to sort the results
    mindate:Optional[int] = None #Initial data to be search from, year
    maxdate:Optional[int] = None #Final data to be search from, year
    idlist:Optional[List[int]]
    email:str = None
    api_key:str = None
    
    @validator('search_tag', always=True)
    def validate_search_tag(cls, v):
        if not v:
            v = 'Title/Abstract'
        if v not in SEARCH_TAGS.keys():
            raise ValueError(f'Search tag need to be some of {SEARCH_TAGS.keys()}')
        return SEARCH_TAGS[v]
    
    @validator('email',always=True)
    def validate_email(cls,v ):
        if 'EMAIL' not in os.environ.keys() or 'API_KEY' not in os.environ.keys():
            print(f"you need to add email and api_key to the environment variables")            
            print(f"['EMAIL'] and ['API_KEY']")
            raise EnvironmentError()
        else: 
            return os.environ.get('EMAIL')
        
    @validator('api_key',always=True)
    def validate_api(cls,v ):
        if 'EMAIL' not in os.environ.keys() or 'API_KEY' not in os.environ.keys():
            print(f"you need to add email and api_key to the environment variables")            
            print(f"['EMAIL'] and ['API_KEY']")
            raise EnvironmentError()
        else: 
            return os.environ.get('API_KEY')


In [ ]:
#| exports

@patch
def search(
    self:Search,
    query: str, #Query to be search in pubmed
):
    """
    It receive a query to be searched in pubmed and return the handler of the search
    """
    Entrez.email = self.email
    Entrez.api_key = self.api_key
    query = query+self.search_tag
    handle = Entrez.esearch(db='pubmed',
                    sort=self.sort,
                    retmax=self.retmax,
                    retmode=self.retmode,
                    term=query,
                    mindate = self.mindate,
                    maxdate =self. maxdate)
    results = Entrez.read(handle)
    return results['IdList']

NameError: name 'patch' is not defined

In [ ]:
search = Search(retmax=2)

In [ ]:
idlist = search.search('divisome')

In [ ]:
idlist

['33536648', '32424210']

In [ ]:
#| export
@patch
def fetch_details(
    self:Search,
    idlist:List[int], #list of pubmedid to be retreived
    ):
    """
    It receive a list of pubmedIds from a search, and retrieve all the details of those publications
    """
    ids = ','.join(idlist)
    handle = Entrez.efetch(db='pubmed',
                           retmode=self.retmode,
                           id=ids)
    results = Entrez.read(handle)
    return results['PubmedArticle']

In [ ]:
#| exports
@patch
def results(
    self:Search,
    query:str, #Term to be queried in pubmed
)->Generator[Result, None, None]:
    """
    Method that do the search and retrieve a generator with all the infomration of the articles"""
    id_list = self.search(query)
    articles = self.fetch_details(id_list)
    for article in articles:
        article_dict = parse_paperinfo(article)
        yield Result.parse_obj(article_dict)

In [ ]:
articles = search.results('bifunctional degraders')

In [ ]:
results = list(articles)

In [ ]:
results[-1].title

'INK4 Tumor Suppressor Proteins Mediate Resistance to CDK4/6 Kinase Inhibitors.'

In [ ]:
results[-1].abstract

'Cyclin-dependent kinases 4 and 6 (CDK4/6) represent a major therapeutic vulnerability for breast cancer. The kinases are clinically targeted via ATP competitive inhibitors (CDK4/6i); however, drug resistance commonly emerges over time. To understand CDK4/6i resistance, we surveyed over 1,300 breast cancers and identified several genetic alterations (e.g., <i>FAT1</i>, <i>PTEN</i>, or <i>ARID1A</i> loss) converging on upregulation of CDK6. Mechanistically, we demonstrate CDK6 causes resistance by inducing and binding CDK inhibitor INK4 proteins (e.g., p18<sup>INK4C</sup>). <i>In vitro</i> binding and kinase assays together with physical modeling reveal that the p18<sup>INK4C</sup>-cyclin D-CDK6 complex occludes CDK4/6i binding while only weakly suppressing ATP binding. Suppression of INK4 expression or its binding to CDK6 restores CDK4/6i sensitivity. To overcome this constraint, we developed bifunctional degraders conjugating palbociclib with E3 ligands. Two resulting lead compounds p

In [ ]:
results[-1].published

datetime.date(2021, 9, 20)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()